In [14]:
import xmltodict
from bs4 import BeautifulSoup
import pprint
import os
import re
import pandas as pd

import skimage
import matplotlib.pyplot as plt

In [15]:
image_folder = os.path.join(os.getcwd(),'IMAGE_DATA')

In [16]:
image_root = os.path.join(image_folder,os.listdir(image_folder)[0])

In [17]:
image_root_lst = os.listdir(image_root)

In [18]:

r = re.compile('(Index)')
# re.findall(r,image_root_lst)
idx_file = list(filter(r.findall,image_root_lst))[0]


In [19]:
with open(os.path.join(image_root,idx_file),'r') as f:
    index_xml = f.read()

In [22]:
# idx_xml = BeautifulSoup(index_xml,'xml')

In [ ]:
# ind_file.find('<Col>'==22)
# ind_file.find_all('Wells', {'Col': [22]})

In [21]:
idx_dict = xmltodict.parse(index_xml)

In [ ]:
# pprint.pprint(idx_dict['EvaluationInputData'])

In [23]:
idx_dict['EvaluationInputData'].keys()

dict_keys(['@Version', '@xmlns', '@xmlns:xsd', '@xmlns:xsi', 'User', 'InstrumentType', 'Plates', 'Wells', 'Maps', 'Images'])

In [24]:
mod_well_lst_to_df = []

for cnt in range(len(idx_dict['EvaluationInputData']['Wells']['Well'])):
    well_temp_df = pd.DataFrame.from_dict(idx_dict['EvaluationInputData']['Wells']['Well'][cnt],orient='columns')
    well_temp_df['Image'] = [item.pop('@id') for item in well_temp_df['Image']]
    mod_well_lst_to_df.append(well_temp_df)
    well_table=pd.concat(mod_well_lst_to_df,axis=0)

In [25]:
well_table.reset_index(drop=True,inplace=True)

In [26]:
well_table.head()

,id,Col,Row,SublayoutID,Image
0,0101,1,1,1,0101K1F1P1R1
1,0101,1,1,1,0101K1F1P1R2
2,0101,1,1,1,0101K1F2P1R1
3,0101,1,1,1,0101K1F2P1R2
4,0101,1,1,1,0101K1F3P1R1


In [27]:
#channelID 1 = HOECHST
#channelID 2 = mCherry
image_table = pd.DataFrame.from_dict(idx_dict['EvaluationInputData']['Images']['Image'],orient='columns')
image_table.head()

,id,State,Row,Col,ChannelID,FieldID,PlaneID,TimepointID,SequenceID,GroupID,AbsTime,AbsPositionZ,MeasurementTimeOffset,URL
0,0101K1F1P1R1,None,1,1,1,1,1,1,1,1,2022-11-15T22:28:38.0496000Z,"{'@Unit': 'm', '#text': '0.1349009528967997'}",0,r01c01-1719728043.tiff
1,0101K1F1P1R2,None,1,1,2,1,1,1,1,1,2022-11-15T22:28:38.6892000Z,"{'@Unit': 'm', '#text': '0.1349009528967997'}",0.639600038528,r01c01-1299190045.tiff
2,0101K1F2P1R1,None,1,1,1,2,1,1,1,1,2022-11-15T22:28:39.4380000Z,"{'@Unit': 'm', '#text': '0.1349040863366522'}",1.38840007782,r01c01-0436032609.tiff
3,0101K1F2P1R2,None,1,1,2,2,1,1,1,1,2022-11-15T22:28:39.9684000Z,"{'@Unit': 'm', '#text': '0.1349040863366522'}",1.91880011559,r01c01-0695440784.tiff
4,0101K1F3P1R1,None,1,1,1,3,1,1,1,1,2022-11-15T22:28:40.8576000Z,"{'@Unit': 'm', '#text': '0.1349020074760486'}",2.80800008774,r01c01-1938356943.tiff


In [28]:
p_ctl_metadata = image_table[image_table['Col']=='22']
p_ctl_metadata.head()

,id,State,Row,Col,ChannelID,FieldID,PlaneID,TimepointID,SequenceID,GroupID,AbsTime,AbsPositionZ,MeasurementTimeOffset,URL
126,0122K1F1P1R1,None,1,22,1,1,1,1,1,1,2022-11-15T22:53:40.6260096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1502.5764091,r01c22-0584869840.tiff
127,0122K1F1P1R2,None,1,22,2,1,1,1,1,1,2022-11-15T22:53:41.1564096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1503.10680914,r01c22-2111396476.tiff
128,0122K1F2P1R1,None,1,22,1,2,1,1,1,1,2022-11-15T22:53:42.4044096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.35480905,r01c22-0165009493.tiff
129,0122K1F2P1R2,None,1,22,2,2,1,1,1,1,2022-11-15T22:53:42.9348096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.88520908,r01c22-2023344791.tiff
130,0122K1F3P1R1,None,1,22,1,3,1,1,1,1,2022-11-15T22:53:43.9020096Z,"{'@Unit': 'm', '#text': '0.13498634681459504'}",1505.85240912,r01c22-1365878811.tiff


In [29]:
p_ctl_metadata['image_root_folder'] = image_root

C:\Users\marfok\AppData\Local\Temp\ipykernel_23224\1373054125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_ctl_metadata['image_root_folder'] = image_root


In [30]:
# get HOESCHT image

In [31]:
p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]['URL'].iloc[0] 

'r01c22-2111396476.tiff'

In [32]:
p_ctl_metadata

,id,State,Row,Col,ChannelID,FieldID,PlaneID,TimepointID,SequenceID,GroupID,AbsTime,AbsPositionZ,MeasurementTimeOffset,URL,image_root_folder
126,0122K1F1P1R1,None,1,22,1,1,1,1,1,1,2022-11-15T22:53:40.6260096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1502.5764091,r01c22-0584869840.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
127,0122K1F1P1R2,None,1,22,2,1,1,1,1,1,2022-11-15T22:53:41.1564096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1503.10680914,r01c22-2111396476.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
128,0122K1F2P1R1,None,1,22,1,2,1,1,1,1,2022-11-15T22:53:42.4044096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.35480905,r01c22-0165009493.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
129,0122K1F2P1R2,None,1,22,2,2,1,1,1,1,2022-11-15T22:53:42.9348096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.88520908,r01c22-2023344791.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
130,0122K1F3P1R1,None,1,22,1,3,1,1,1,1,2022-11-15T22:53:43.9020096Z,"{'@Unit': 'm', '#text': '0.13498634681459504'}",1505.85240912,r01c22-1365878811.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,1622K1F1P1R2,None,16,22,2,1,1,1,1,1,2022-11-15T22:52:34.5912092Z,"{'@Unit': 'm', '#text': '0.13489568258617993'}",1436.54160905,r16c22-1657699482.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
2288,1622K1F2P1R1,None,16,22,1,2,1,1,1,1,2022-11-15T22:52:35.8548092Z,"{'@Unit': 'm', '#text': '0.13489894448944267'}",1437.80520916,r16c22-1734411264.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
2289,1622K1F2P1R2,None,16,22,2,2,1,1,1,1,2022-11-15T22:52:36.3852092Z,"{'@Unit': 'm', '#text': '0.13489894448944267'}",1438.3356092,r16c22-0524236398.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
2290,1622K1F3P1R1,None,16,22,1,3,1,1,1,1,2022-11-15T22:52:37.2588092Z,"{'@Unit': 'm', '#text': '0.13489325815789818'}",1439.2092092,r16c22-0431998733.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...


In [35]:
im1 = os.path.join(p_ctl_metadata['image_root_folder'].iloc[0],p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]['URL'].iloc[0] )
plt.imshow(skimage.io.imread(im1))
plt.title('mCherry')

ValueError: <COMPRESSION.LZW: 5> requires the 'imagecodecs' package

In [37]:
len(p_ctl_metadata['image_root_folder'])

96

In [38]:
len(p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]['URL'])

48

In [39]:
trim_pctl = p_ctl_metadata[['id','Row','Col','ChannelID','FieldID','URL','image_root_folder']]
h_df = trim_pctl[trim_pctl.ChannelID.eq('1')] 
m_df = trim_pctl[trim_pctl.ChannelID.eq('2')]

In [40]:
# 16 total images
display(h_df.head())
print(h_df.shape)

,id,Row,Col,ChannelID,FieldID,URL,image_root_folder
126,0122K1F1P1R1,1,22,1,1,r01c22-0584869840.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
128,0122K1F2P1R1,1,22,1,2,r01c22-0165009493.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
130,0122K1F3P1R1,1,22,1,3,r01c22-1365878811.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
270,0222K1F1P1R1,2,22,1,1,r02c22-1656980324.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
272,0222K1F2P1R1,2,22,1,2,r02c22-1027735946.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...


(48, 7)


In [41]:
display(m_df.head())
print(m_df.shape)

,id,Row,Col,ChannelID,FieldID,URL,image_root_folder
127,0122K1F1P1R2,1,22,2,1,r01c22-2111396476.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
129,0122K1F2P1R2,1,22,2,2,r01c22-2023344791.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
131,0122K1F3P1R2,1,22,2,3,r01c22-0711364633.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
271,0222K1F1P1R2,2,22,2,1,r02c22-2137399416.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...
273,0222K1F2P1R2,2,22,2,2,r02c22-0509830294.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...


(48, 7)


In [42]:
p_ctl_metadata.shape

(96, 15)

In [43]:
lst = []
for i in range(p_ctl_metadata.shape[0]//2):
    # print('mCherry image path: ', os.path.join(p_ctl_metadata['image_root_folder'].iloc[i],p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]['URL'].iloc[i]))
    # print('HOECHST image path: ', os.path.join(p_ctl_metadata['image_root_folder'].iloc[i],p_ctl_metadata[p_ctl_metadata.ChannelID.eq('1')]['URL'].iloc[i]))
    im1 = os.path.join(p_ctl_metadata['image_root_folder'].iloc[i],p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]['URL'].iloc[i])
    im2 = os.path.join(p_ctl_metadata['image_root_folder'].iloc[i],p_ctl_metadata[p_ctl_metadata.ChannelID.eq('1')]['URL'].iloc[i])
    lst.append(im1)

In [44]:
len(lst)

48

In [45]:
p_ctl_metadata['compound'] = 'POS-000001'

C:\Users\marfok\AppData\Local\Temp\ipykernel_23224\1735611418.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_ctl_metadata['compound'] = 'POS-000001'


In [46]:
p_ctl_metadata['compound_uM'] = 'uM'

C:\Users\marfok\AppData\Local\Temp\ipykernel_23224\2743659340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_ctl_metadata['compound_uM'] = 'uM'


In [47]:
p_ctl_metadata['table_nr'] = 'Blank'

C:\Users\marfok\AppData\Local\Temp\ipykernel_23224\2764959845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_ctl_metadata['table_nr'] = 'Blank'


In [48]:
p_ctl_metadata.reset_index(drop=True,inplace=True)

In [49]:
p_ctl_metadata.loc[0, 'compound']

'POS-000001'

In [50]:
j =[]
for i,row in p_ctl_metadata.iterrows():
    j.append(p_ctl_metadata.loc[i,'compound'])
       

In [51]:
p_ctl_metadata['pseudoclass'] = ['class_{}'.format(i) for i in range(p_ctl_metadata.shape[0])]

C:\Users\marfok\AppData\Local\Temp\ipykernel_23224\1450283836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_ctl_metadata['pseudoclass'] = ['class_{}'.format(i) for i in range(p_ctl_metadata.shape[0])]


In [52]:
p_ctl_metadata.head()

,id,State,Row,Col,ChannelID,FieldID,PlaneID,TimepointID,SequenceID,GroupID,AbsTime,AbsPositionZ,MeasurementTimeOffset,URL,image_root_folder,compound,compound_uM,table_nr,pseudoclass
0,0122K1F1P1R1,None,1,22,1,1,1,1,1,1,2022-11-15T22:53:40.6260096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1502.5764091,r01c22-0584869840.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_0
1,0122K1F1P1R2,None,1,22,2,1,1,1,1,1,2022-11-15T22:53:41.1564096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1503.10680914,r01c22-2111396476.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_1
2,0122K1F2P1R1,None,1,22,1,2,1,1,1,1,2022-11-15T22:53:42.4044096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.35480905,r01c22-0165009493.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_2
3,0122K1F2P1R2,None,1,22,2,2,1,1,1,1,2022-11-15T22:53:42.9348096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.88520908,r01c22-2023344791.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_3
4,0122K1F3P1R1,None,1,22,1,3,1,1,1,1,2022-11-15T22:53:43.9020096Z,"{'@Unit': 'm', '#text': '0.13498634681459504'}",1505.85240912,r01c22-1365878811.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_4


In [58]:
p_ctl_metadata

,id,State,Row,Col,ChannelID,FieldID,PlaneID,TimepointID,SequenceID,GroupID,AbsTime,AbsPositionZ,MeasurementTimeOffset,URL,image_root_folder,compound,compound_uM,table_nr,pseudoclass
0,0122K1F1P1R1,None,1,22,1,1,1,1,1,1,2022-11-15T22:53:40.6260096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1502.5764091,r01c22-0584869840.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_0
1,0122K1F1P1R2,None,1,22,2,1,1,1,1,1,2022-11-15T22:53:41.1564096Z,"{'@Unit': 'm', '#text': '0.13498944511922153'}",1503.10680914,r01c22-2111396476.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_1
2,0122K1F2P1R1,None,1,22,1,2,1,1,1,1,2022-11-15T22:53:42.4044096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.35480905,r01c22-0165009493.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_2
3,0122K1F2P1R2,None,1,22,2,2,1,1,1,1,2022-11-15T22:53:42.9348096Z,"{'@Unit': 'm', '#text': '0.134989877075187'}",1504.88520908,r01c22-2023344791.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_3
4,0122K1F3P1R1,None,1,22,1,3,1,1,1,1,2022-11-15T22:53:43.9020096Z,"{'@Unit': 'm', '#text': '0.13498634681459504'}",1505.85240912,r01c22-1365878811.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1622K1F1P1R2,None,16,22,2,1,1,1,1,1,2022-11-15T22:52:34.5912092Z,"{'@Unit': 'm', '#text': '0.13489568258617993'}",1436.54160905,r16c22-1657699482.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_91
92,1622K1F2P1R1,None,16,22,1,2,1,1,1,1,2022-11-15T22:52:35.8548092Z,"{'@Unit': 'm', '#text': '0.13489894448944267'}",1437.80520916,r16c22-1734411264.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_92
93,1622K1F2P1R2,None,16,22,2,2,1,1,1,1,2022-11-15T22:52:36.3852092Z,"{'@Unit': 'm', '#text': '0.13489894448944267'}",1438.3356092,r16c22-0524236398.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_93
94,1622K1F3P1R1,None,16,22,1,3,1,1,1,1,2022-11-15T22:52:37.2588092Z,"{'@Unit': 'm', '#text': '0.13489325815789818'}",1439.2092092,r16c22-0431998733.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_94


In [88]:
# p_ctl_metadata.columns[1:]

In [86]:
# p_ctl_metadata[p_ctl_metadata.ChannelID.eq('1')]['URL']

In [87]:
# pd.pivot(p_ctl_metadata,index=None,columns='id',values = )

In [38]:
p_ctl_metadata.columns

Index(['id', 'State', 'Row', 'Col', 'ChannelID', 'FieldID', 'PlaneID',
       'TimepointID', 'SequenceID', 'GroupID', 'AbsTime', 'AbsPositionZ',
       'MeasurementTimeOffset', 'URL', 'image_root_folder', 'compound',
       'compound_uM', 'table_nr', 'pseudoclass'],
      dtype='object')

In [ ]:
p_ctl_metadata.to_csv('sample_data.csv')

#### create new dataframe with type of cell stain as a column

In [276]:
h_df = p_ctl_metadata[p_ctl_metadata.ChannelID.eq('1')]
m_df = p_ctl_metadata[p_ctl_metadata.ChannelID.eq('2')]

h = h_df
m = m_df[['id','Row','Col','URL','ChannelID']]
print(h.shape)
print(m.shape)

(48, 19)
(48, 5)


In [277]:
mer_df = h.merge(m.drop_duplicates(['Row','Col']),\
               on=['Row','Col'],\
              how='inner')

In [278]:
mer_df.shape

(48, 22)

In [279]:
mer_df.columns

Index(['id_x', 'State', 'Row', 'Col', 'ChannelID_x', 'FieldID', 'PlaneID',
       'TimepointID', 'SequenceID', 'GroupID', 'AbsTime', 'AbsPositionZ',
       'MeasurementTimeOffset', 'URL_x', 'image_root_folder', 'compound',
       'compound_uM', 'table_nr', 'pseudoclass', 'id_y', 'URL_y',
       'ChannelID_y'],
      dtype='object')

In [282]:
mer_df.rename(columns={'URL_x':'HOECHST_IMAGE','URL_y':'mCHERRY_IMAGE',
                       'id_x':'id_HOECHST', 'id_y':'id_mCHERRY',
                       'ChannelID_x':'ChannelID_HOECHST', 'ChannelID_y':'ChannelID_mCHERRY'},inplace=True)

In [288]:
mer_df['plate'] = 'PTL-000001'

In [289]:
mer_df.columns

Index(['id_HOECHST', 'State', 'Row', 'Col', 'ChannelID_HOECHST', 'FieldID',
       'PlaneID', 'TimepointID', 'SequenceID', 'GroupID', 'AbsTime',
       'AbsPositionZ', 'MeasurementTimeOffset', 'HOECHST_IMAGE',
       'image_root_folder', 'compound', 'compound_uM', 'table_nr',
       'pseudoclass', 'id_mCHERRY', 'mCHERRY_IMAGE', 'ChannelID_mCHERRY',
       'plate'],
      dtype='object')

In [290]:
mer_df.head()

,id_HOECHST,State,Row,Col,ChannelID_HOECHST,FieldID,PlaneID,TimepointID,SequenceID,GroupID,...,HOECHST_IMAGE,image_root_folder,compound,compound_uM,table_nr,pseudoclass,id_mCHERRY,mCHERRY_IMAGE,ChannelID_mCHERRY,plate
0,0122K1F1P1R1,None,1,22,1,1,1,1,1,1,...,r01c22-0584869840.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_0,0122K1F1P1R2,r01c22-2111396476.tiff,2,PTL-000001
1,0122K1F2P1R1,None,1,22,1,2,1,1,1,1,...,r01c22-0165009493.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_2,0122K1F1P1R2,r01c22-2111396476.tiff,2,PTL-000001
2,0122K1F3P1R1,None,1,22,1,3,1,1,1,1,...,r01c22-1365878811.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_4,0122K1F1P1R2,r01c22-2111396476.tiff,2,PTL-000001
3,0222K1F1P1R1,None,2,22,1,1,1,1,1,1,...,r02c22-1656980324.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_6,0222K1F1P1R2,r02c22-2137399416.tiff,2,PTL-000001
4,0222K1F2P1R1,None,2,22,1,2,1,1,1,1,...,r02c22-1027735946.tiff,c:\Users\marfok\OneDrive - Bristol Myers Squib...,POS-000001,uM,Blank,class_8,0222K1F1P1R2,r02c22-2137399416.tiff,2,PTL-000001


In [287]:
mer_df.to_csv('pivoted_channelID_data.csv')

In [ ]:


# # r = re.compile('.*r04c11')
# r = re.compile('(Index)')

# with open (list(filter(r.findall,im_lst))[0],'r') as f:
#     z = f.read()
# z


# import numpy as np
# samp = np.array([21,22,33,4,5,6,7,87])
# mask = np.array([0,0,0,1,1,1,1,0],dtype=bool)
# samp[mask]

